In [ ]:
import requests
import json

from flask import Flask, render_template
from flask import request

In [ ]:
def rendering(query = None, category = None):
    print('rendering:',query, category)
    if query is not None:
        print('query isnt none')
        response_checker = requests.post('http://127.0.0.1:13539/spellchecker',
                                         json={'query': query})
        parsed_checker = json.loads(response_checker.text)

        if parsed_checker['state']:
            # if checker found mistakes in the query
            r = requests.post('http://127.0.0.1:13565/search', 
                              json={'query': parsed_checker["processed_data"], 
                                    'category': category,
                                    'max_docs': 10}).json()["documents"]
            if len(r) == 0:
                return render_template('blank.html', old_query=query)
            title = [doc['title'] for doc in r]
            url = [doc['url'] for doc in r]
            snippet = [doc['snippet'] +' (' + str(doc['id']) +')' for doc in r]

            return render_template('checked.html', old_query=query,
                                   checked_query=parsed_checker["processed_data"],
                                   title=title, url=url, snippet=snippet)
        else:
            r = requests.post('http://127.0.0.1:13565/search', 
                              json={'query': query, 'category': category, 
                                    'max_docs': 10}).json()["documents"]
            if len(r) == 0:
                return render_template('blank.html', old_query=query)
            title = [doc['title'] for doc in r]
            url = [doc['url'] for doc in r]
            snippet = [doc['snippet'] +' (' + str(doc['id']) +')' for doc in r]

            return render_template('result.html', old_query=query, 
                                   query=query,title=title, url=url, 
                                   snippet=snippet)
        
    else:
        print('query is none')
        r = requests.post('http://127.0.0.1:13565/search', 
                              json={'query': None, 'category': category,
                                    'max_docs': 10}).json()["documents"]
        
        title = [doc['title'] for doc in r]
        url = [doc['url'] for doc in r]
        snippet = [doc['snippet'] +' (' + str(doc['id']) +')' for doc in r]

        return render_template('result.html', old_query=query,
                               checked_query=parsed_checker["processed_data"],
                               title=title, url=url, snippet=snippet)

    

In [ ]:
'''
Client with web UI
'''
app = Flask(__name__)
  

@app.route('/')
def index():
    return render_template('search.html')

@app.route('/search', methods=['POST'])
def search():
    
    formData = request.values if request.method == "POST" else request.values
    query = formData['query']
    category = formData['category']   
    print(query, category)
    
    if not query and category == 'none':
        print ("Nothing")
    if query and category == 'none':
        print("Only query")
    if not query and category != 'none':
        print("Only category")
    if query and category != 'none':
        print("Query and category")
    
    response_checker = requests.post('http://127.0.0.1:13539/spellchecker',
                                     json={'query': query})
    parsed_checker = json.loads(response_checker.text)

    if parsed_checker['state']:
        # if checker found mistakes in the query
        r = requests.post('http://127.0.0.1:13565/search', 
                          json={'query': parsed_checker["processed_data"], 
                                'max_docs': 10}).json()["documents"]
        if len(r) == 0:
            return render_template('blank.html', old_query=query, form=form)
        title = [doc['title'] for doc in r]
        url = [doc['url'] for doc in r]
        snippet = [doc['snippet'] +' (' + str(doc['id']) +')' for doc in r]

        return render_template('checked.html', old_query=query,
                               checked_query=parsed_checker["processed_data"],
                               title=title, url=url, snippet=snippet, form=form)
    else:
        r = requests.post('http://127.0.0.1:13565/search', 
                          json={'query': query, 'max_docs': 10}).json()["documents"]
        if len(r) == 0:
            return render_template('blank.html', old_query=query, form=form)

        title = [doc['title'] for doc in r]
        url = [doc['url'] for doc in r]
        snippet = [doc['snippet'] +' (' + str(doc['id']) +')' for doc in r]

        return render_template('result.html', old_query=query, 
                               query=query,title=title, url=url, 
                               snippet=snippet, form=form)

    return render_template('search.html', form=form)

if __name__ == '__main__':
    app.run(host='127.0.0.1', port=13560)

In [ ]:
app = Flask(__name__)
  

@app.route('/')
def index():
    return render_template('search.html')

@app.route('/search', methods=['POST'])
def search():
    
    formData = request.values if request.method == "POST" else request.values
    query = formData['query']
    category = formData['category']   
    print('query: ', query, 'category: ', category)
    
    if not query and category == 'none':
        return render_template('blank.html', old_query=query)
    else:
        if query and category == 'none':
            result = rendering(query=query)        
        if not query and category != 'none':
            result = rendering(category=category)
        if query and category != 'none':
            result = rendering(query=query, category=category)    
    
    return result

if __name__ == '__main__':
    app.run(host='127.0.0.1', port=13560)